# NBA Most Valuable of the Year
## Data Science Final Project

Group Member 1: David Basin
Group Member 2: Mateo Castro
Group Member 3: Abed Islam
***

We will be attempting to determine who will win the NBA Most Valuable Player of the Year award using machine learning methods such as SVM/Linear Regression and K-Folds. Our findings will be based on the advanced analystics of past MVP winners, their teams and the advanced analytics of this years players. The datasets to be used are provided.

### Part 1: EDA 
In this part, we will be performing Exploratory Data Analysis on the raw datasets to provide some inital insights to the data.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm, linear_model, metrics
%matplotlib inline 



### Part 2: Cleaning the data
We will be cleaning the data we have so that we have only the information we will need for our models.

In [51]:
for i in range(77,121):
    years = str(i)[-2:]+"-"+str(i+1)[-2:]
    voting = pd.read_csv("Voting/"+years+"_Voting.csv", encoding = "ISO-8859-1", engine="python")
    advanced = pd.read_csv("advanced_stats/"+years+"_advanced.csv", encoding = "ISO-8859-1", engine="python")
    pergame = pd.read_csv("PerGame/PerGame_"+years+".csv", encoding = "ISO-8859-1", engine="python")
    #per game: FG%, TRB, AST, STL, BLK, TOV, PTS
    #advanced: PER, TS%, USG%, WS, OBPM, DBPM, VORP
    #Voting: Tm, Share
    voting = voting.loc[:,["Player","Tm","Share"]]
    advanced = advanced.loc[:,["Player","PER","TS%","USG%","WS","OBPM","DBPM","VORP"]]
    advanced["Player"] = advanced["Player"].str.replace('*','')
    pergame = pergame.loc[:,["Player","FG%","TRB","AST","STL","BLK","TOV","PTS"]]
    pergame["Player"] = pergame["Player"].str.replace('*','')
    final_df = pd.merge(voting, advanced, how="left", on="Player")
    final_df = pd.merge(final_df, pergame, how="left", on="Player")
    final_df["Year"] = years
    print(final_df)
advanced_22 = pd.read_csv("advanced_stats/21-22_advanced.csv", encoding = "ISO-8859-1", engine="python")
pergame_22 = pd.read_csv("PerGame/PerGame_21-22.csv", encoding = "ISO-8859-1", engine="python")
advanced_22 = advanced.loc[:,["Player","PER","TS%","USG%","WS","OBPM","DBPM","VORP"]]
advanced_22["Player"] = advanced["Player"].str.replace('*','')
pergame_22 = pergame.loc[:,["Player","FG%","TRB","AST","STL","BLK","TOV","PTS"]]
pergame_22["Player"] = pergame["Player"].str.replace('*','')
final_df_22 = pd.merge(advanced_22, pergame_22, how="left", on="Player")
print(final_df_22)

C:\Users\dbasi\AppData\Local\Temp/ipykernel_12704/4036091920.py:11: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  advanced["Player"] = advanced["Player"].str.replace('*','')
C:\Users\dbasi\AppData\Local\Temp/ipykernel_12704/4036091920.py:13: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  pergame["Player"] = pergame["Player"].str.replace('*','')
C:\Users\dbasi\AppData\Local\Temp/ipykernel_12704/4036091920.py:11: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  advanced["Player"] = advanced["Player"].str.replace('*','')
C:\Use

                 Player   Tm  Share   PER    TS%  USG%    WS  OBPM  DBPM  \
0           Bill Walton  POR  0.403  24.8  0.554  24.5   8.4   4.4   3.7   
1         George Gervin  SAS  0.338  24.7  0.594  29.5  12.0   4.8   0.0   
2        David Thompson  DEN  0.120  23.2  0.578  26.8  12.7   5.5  -0.7   
3   Kareem Abdul-Jabbar  LAL  0.059  29.2  0.589  27.0  12.1   6.5   2.9   
4          Walter Davis  PHO  0.017  22.1  0.577  28.5  10.1   3.7  -0.4   
5         Maurice Lucas  POR  0.013  17.3  0.502  24.2   5.8   0.7  -0.1   
6         Paul Westphal  PHO  0.013  23.8  0.565  30.9   9.8   4.7   0.0   
7         Artis Gilmore  CHI  0.008  23.5  0.604  25.3  11.5   3.8   0.3   
8        Truck Robinson  NOJ  0.008  15.7  0.481  23.3   6.5  -0.5  -0.9   
9         Julius Erving  PHI  0.004  21.7  0.555  25.1   9.1   4.0   0.7   
10      Marques Johnson  MIL  0.004  21.3  0.563  21.4  10.6   3.3  -0.5   
11           Bob Lanier  DET  0.004  24.8  0.580  25.7  10.1   4.7   0.7   
12          

C:\Users\dbasi\AppData\Local\Temp/ipykernel_12704/4036091920.py:11: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  advanced["Player"] = advanced["Player"].str.replace('*','')
C:\Users\dbasi\AppData\Local\Temp/ipykernel_12704/4036091920.py:13: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  pergame["Player"] = pergame["Player"].str.replace('*','')
C:\Users\dbasi\AppData\Local\Temp/ipykernel_12704/4036091920.py:11: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  advanced["Player"] = advanced["Player"].str.replace('*','')
C:\Use

                 Player   Tm  Share   PER    TS%  USG%    WS  OBPM  DBPM  \
0          Moses Malone  HOU  0.735  26.8  0.576  29.9  15.4   6.2  -1.8   
1            Larry Bird  BOS  0.588  22.6  0.557  25.2  12.5   4.4   2.3   
2         Julius Erving  PHI  0.294  25.9  0.593  28.1  13.3   6.0   1.7   
3         Robert Parish  BOS  0.190  22.6  0.571  25.5  10.0   2.6   1.2   
4          Gus Williams  SEA  0.167  21.8  0.526  28.5  10.2   4.1   1.0   
5         George Gervin  SAS  0.159  24.2  0.562  35.0  10.7   5.4  -1.5   
6       Sidney Moncrief  MIL  0.146  20.1  0.601  21.0  13.4   3.9   0.9   
7         Magic Johnson  LAL  0.097  22.9  0.590  20.1  12.9   4.9   2.3   
8            Jack Sikma  SEA  0.048  20.6  0.559  22.5  12.6   2.2   1.7   
9   Kareem Abdul-Jabbar  LAL  0.045  23.4  0.608  25.6  10.7   3.8   1.2   
10       Dan Roundfield  ATL  0.020  19.5  0.527  24.6   6.5   1.6   0.7   
11       Kelly Tripucka  DET  0.020  16.4  0.570  23.3   7.5   1.5  -1.3   
12         A

C:\Users\dbasi\AppData\Local\Temp/ipykernel_12704/4036091920.py:11: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  advanced["Player"] = advanced["Player"].str.replace('*','')
C:\Users\dbasi\AppData\Local\Temp/ipykernel_12704/4036091920.py:13: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  pergame["Player"] = pergame["Player"].str.replace('*','')
C:\Users\dbasi\AppData\Local\Temp/ipykernel_12704/4036091920.py:11: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  advanced["Player"] = advanced["Player"].str.replace('*','')
C:\Use

...

C:\Users\dbasi\AppData\Local\Temp/ipykernel_12704/4036091920.py:11: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  advanced["Player"] = advanced["Player"].str.replace('*','')
C:\Users\dbasi\AppData\Local\Temp/ipykernel_12704/4036091920.py:13: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  pergame["Player"] = pergame["Player"].str.replace('*','')
C:\Users\dbasi\AppData\Local\Temp/ipykernel_12704/4036091920.py:11: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  advanced["Player"] = advanced["Player"].str.replace('*','')
C:\Use

              Player   Tm  Share   PER    TS%  USG%    WS  OBPM  DBPM  VORP  \
0      Stephen Curry  GSW  1.000  31.5  0.669  32.6  17.9  10.3   1.6   9.5   
1      Kawhi Leonard  SAS  0.484  26.0  0.616  25.8  13.7   6.1   3.0   6.7   
2       LeBron James  CLE  0.482  27.5  0.588  31.4  13.6   7.0   2.0   7.5   
3  Russell Westbrook  OKC  0.371  27.6  0.554  31.6  14.0   6.4   1.4   6.8   
4       Kevin Durant  OKC  0.112  28.2  0.634  30.6  14.5   8.4   1.6   7.8   
5         Chris Paul  LAC  0.082  26.2  0.575  27.1  12.7   5.9   2.0   6.0   
6     Draymond Green  GSW  0.038  19.3  0.587  18.8  11.1   2.6   2.9   5.3   
7     Damian Lillard  POR  0.020  22.2  0.560  31.3   9.2   5.7  -1.3   4.3   
8       James Harden  HOU  0.007  25.3  0.598  32.5  13.3   6.4   0.2   6.8   
9         Kyle Lowry  TOR  0.005  22.2  0.578  26.1  11.6   4.7   1.1   5.7   

     FG%  TRB   AST  STL  BLK  TOV   PTS   Year  
0  0.504  5.4   6.7  2.1  0.2  3.3  30.1  15-16  
1  0.506  6.8   2.6  1.8  1.0 

C:\Users\dbasi\AppData\Local\Temp/ipykernel_12704/4036091920.py:11: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  advanced["Player"] = advanced["Player"].str.replace('*','')
C:\Users\dbasi\AppData\Local\Temp/ipykernel_12704/4036091920.py:13: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  pergame["Player"] = pergame["Player"].str.replace('*','')
C:\Users\dbasi\AppData\Local\Temp/ipykernel_12704/4036091920.py:11: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  advanced["Player"] = advanced["Player"].str.replace('*','')
C:\Use

                   Player   Tm  Share   PER    TS%  USG%    WS  OBPM  DBPM  \
0   Giannis Antetokounmpo  MIL  0.932  30.9  0.644  32.3  14.4   6.3   4.1   
1            James Harden  HOU  0.768  30.6  0.616  40.5  15.2   9.4   1.6   
2             Paul George  OKC  0.352  23.3  0.583  29.5  11.9   5.6   1.6   
3           Nikola JokiÄ  DEN  0.210  26.3  0.589  27.4  11.8   6.2   2.9   
4           Stephen Curry  GSW  0.173  24.4  0.641  30.4   9.7   7.1  -0.5   
5          Damian Lillard  POR  0.068  23.7  0.588  29.3  12.1   6.6  -0.3   
6             Joel Embiid  PHI  0.049  26.1  0.593  33.3   8.7   4.0   0.9   
7            Kevin Durant  GSW  0.025  24.2  0.631  29.0  11.5   5.4   0.1   
8           Kawhi Leonard  TOR  0.013  25.8  0.606  30.3   9.5   6.1   1.1   
9       Russell Westbrook  OKC  0.008  21.1  0.501  30.9   6.8   2.8   2.4   
10            Rudy Gobert  UTA  0.001  24.6  0.682  17.8  14.4   3.0   2.5   
11           LeBron James  LAL  0.001  25.6  0.588  31.6   7.2  

C:\Users\dbasi\AppData\Local\Temp/ipykernel_12704/4036091920.py:21: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  advanced_22["Player"] = advanced["Player"].str.replace('*','')
C:\Users\dbasi\AppData\Local\Temp/ipykernel_12704/4036091920.py:23: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  pergame_22["Player"] = pergame["Player"].str.replace('*','')


## Part 3: Creating and training the model
Creating the model and training it with the train set.

## Part 4: Validating and testing the model
We will optimize the model using K-Folds validation to find the best parameters to use and testing the resulting model.

## Part 5: Determining the MVP
Finally, the MVP will be determined for this season using the model created.